# Playing with NOvA HDF5 Files
# How easy is it to create some quick and dirty FD MC analysis plots



In [2]:
import h5py
import numpy as np
import matplotlib.pyplot as plt 
import math

In [3]:
import enum 

# Neutrino interaction categories
class Mode(enum.Enum): 
    kUnknownMode               = -1
    kQE                        = 0
    kRes                       = 1
    kDIS                       = 2
    kCoh                       = 3
    kCohElastic                = 4
    kElectronScattering        = 5
    kIMDAnnihilation           = 6
    kInverseBetaDecay          = 7
    kGlashowResonance          = 8
    kAMNuGamma                 = 9
    kMEC                       = 10
    kDiffractive               = 11
    kEM                        = 12
    kWeakMix                   = 13



class Interaction(enum.Enum):
    kNumuQE =0           # Numu CC QE interaction
    kNumuRes =1           # Numu CC Resonant interaction
    kNumuDIS = 2          # Numu CC DIS interaction
    kNumuOther = 3        # Numu CC, other than above
    kNueQE = 4            # Nue CC QE interaction
    kNueRes = 5           # Nue CC Resonant interaction
    kNueDIS = 6           # Nue CC DIS interaction
    kNueOther = 7         # Nue CC, other than above
    kNutauQE = 8          # Nutau CC QE interaction
    kNutauRes = 9         # Nutau CC Resonant interaction
    kNutauDIS =10         # Nutau CC DIS interaction
    kNutauOther =11       # Nutau CC, other than above
    kNuElectronElastic = 12# NC Nu On E Scattering
    kNC =13                # NC interaction
    kCosmic =14           # Cosmic ray background
    kOther =15            # Something else.  Tau?  Hopefully we don't use this
    kNIntType=16          # Number of interaction types, used like a vector size

    
class FinalState(enum.Enum):
    kNumu0tr0sh=0          # Numu CC - no track no shower
    kNumu0tr1sh=1          # Numu CC - no track  1 shower
    kNumu0tr2sh=enum.auto()          # Numu CC - no track  2 shower
    kNumu0trMsh=enum.auto()          # Numu CC - no track 3+ shower
    kNumu1tr0sh=enum.auto()          # Numu CC -  1 track no shower
    kNumu1tr1sh=enum.auto()          # Numu CC -  1 track  1 shower
    kNumu1tr2sh=enum.auto()          # Numu CC -  1 track  2 shower
    kNumu1trMsh=enum.auto()          # Numu CC -  1 track 3+ shower
    kNumu2tr0sh=enum.auto()          # Numu CC -  2 track no shower
    kNumu2tr1sh=enum.auto()          # Numu CC -  2 track  1 shower
    kNumu2tr2sh=enum.auto()          # Numu CC -  2 track  2 shower
    kNumu2trMsh=enum.auto()          # Numu CC -  2 track 3+ shower
    kNumuMtr0sh=enum.auto()          # Numu CC - 3+ track no showe
    kNumuMtr1sh=enum.auto()          # Numu CC - 3+ track  1 shower
    kNumuMtr2sh=enum.auto()          # Numu CC - 3+ track  2 showe
    kNumuMtrMsh=enum.auto()          # Numu CC - 3+ track 3+ shower
    kNue0tr0sh=enum.auto()           # Nue CC - no track no shower
    kNue0tr1sh=enum.auto()           # Nue CC - no track  1 shower
    kNue0tr2sh=enum.auto()           # Nue CC - no track  2 showe
    kNue0trMsh=enum.auto()           # Nue CC - no track 3+ shower
    kNue1tr0sh=enum.auto()           # Nue CC -  1 track no shower
    kNue1tr1sh=enum.auto()           # Nue CC -  1 track  1 shower
    kNue1tr2sh=enum.auto()           # Nue CC -  1 track  2 shower
    kNue1trMsh=enum.auto()           # Nue CC -  1 track 3+ shower
    kNue2tr0sh=enum.auto()           # Nue CC -  2 track no shower
    kNue2tr1sh=enum.auto()           # Nue CC -  2 track  1 shower
    kNue2tr2sh=enum.auto()           # Nue CC -  2 track  2 shower
    kNue2trMsh=enum.auto()           # Nue CC -  2 track 3+ shower
    kNueMtr0sh=enum.auto()           # Nue CC - 3+ track no shower
    kNueMtr1sh=enum.auto()           # Nue CC - 3+ track  1 shower
    kNueMtr2sh=enum.auto()           # Nue CC - 3+ track  2 shower
    kNueMtrMsh=enum.auto()           # Nue CC - 3+ track 3+ shower
    kNC0tr0sh=enum.auto()           # NC CC - no track no shower
    kNC0tr1sh=enum.auto()           # NC CC - no track  1 shower
    kNC0tr2sh=enum.auto()           # NC CC - no track  2 shower
    kNC0trMsh=enum.auto()           # NC CC - no track 3+ shower
    kNC1tr0sh=enum.auto()           # NC CC -  1 track no shower
    kNC1tr1sh=enum.auto()           # NC CC -  1 track  1 shower
    kNC1tr2sh=enum.auto()           # NC CC -  1 track  2 shower
    kNC1trMsh=enum.auto()           # NC CC -  1 track 3+ shower
    kNC2tr0sh=enum.auto()           # NC CC -  2 track no shower
    kNC2tr1sh=enum.auto()           # NC CC -  2 track  1 shower
    kNC2tr2sh=enum.auto()           # NC CC -  2 track  2 shower
    kNC2trMsh=enum.auto()           # NC CC -  2 track 3+ shower
    kNCMtr0sh=enum.auto()           # NC CC - 3+ track no shower
    kNCMtr1sh=enum.auto()           # NC CC - 3+ track  1 shower
    kNCMtr2sh=enum.auto()           # NC CC - 3+ track  2 shower
    kNCMtrMsh=enum.auto()           # NC CC - 3+ track 3+ shower
    kCosmicFS=enum.auto()           # Cosmic ray background
    kOtherFS=enum.auto()            # Something else.  Tau?  Hopefully we don't use this
    kNFStType=enum.auto()            # Number of final state types, used like a vector size



In [10]:
# import glob
import glob
 
# Copy a network object to a local file
#urllib.request.urlretrieve('http://www.hep.ucl.ac.uk/undergrad/0056/other/projects/nova/neutrino1.h5', "neutrino1.h5")

#Open the local h5 file with h5py
#fileName='FD-Nominal-FHC-Nonswap/fardet_genie_N1810j0211a_nonswap_genierw_fhc_v08_1000_r00035865_s12_c000_R20-11-25-prod5.1reco.l_v1_20211112_171447_sim.h5caf.h5'
#df=h5py.File('FD-Nominal-FHC-Nonswap/fardet_genie_N1810j0211a_nonswap_genierw_fhc_v08_1000_r00035865_s12_c000_R20-11-25-prod5.1reco.l_v1_20211112_171447_sim.h5caf.h5','r')
#df=h5py.File('mergecopy.h5','r')
#df=h5py.File('FD-Nominal-FHC-Nonswap_copymerge/copymerged_c14_cl100.h5')
df=h5py.File('FD-Nominal-FHC-Nonswap_copymerge/merged_copymerge_scann.h5')    



In [11]:
#Print the keys in the neutrino meta data
print(df.keys())
#print(df['rec'].keys())
print('rec.mc.nu',df['rec.mc.nu'].keys())
print('rec.mc.nu.beam',df['rec.mc.nu.beam'].keys())
print('rec.energy.numu',df['rec.energy.numu'].keys())
print('rec.energy.numu.hadclust',df['rec.energy.numu.hadclust'].keys())
#print('rec.trk.kalman',df['rec.trk.kalman'].keys())
#print('rec.trk.kalman.tracks',df['rec.trk.kalman.tracks'].keys())
print('rec.sel.remid',df['rec.sel.remid'].keys())
print('rec.sel.cosrej',df['rec.sel.cosrej'].keys())
print('rec.sel.cvnloosepreselptp',df['rec.sel.cvnloosepreselptp'].keys())
print('rec.training.trainingdata',df['rec.training.trainingdata'].keys())
print('rec.spill.cosmiccvn',df['rec.spill.cosmiccvn'].keys())

cvnmaps=df['rec.training.cvnmaps']
print(cvnmaps['cvnmap'])

#Get an numpy array containing the event image, and reshape it from flat to 2x100x80
print(np.shape(df['rec.training.cvnmaps/cvnmap']))
print("Got some events:",np.shape(df['rec.training.cvnmaps/cvnmap'])[0])
print(df['rec.training.trainingdata']['evt'])
event0=np.array(df['rec.training.cvnmaps/cvnmap'][7]).reshape((2,100,80))

<KeysViewHDF5 ['rec.energy.numu', 'rec.energy.numu.hadclust', 'rec.mc', 'rec.mc.nu', 'rec.mc.nu.beam', 'rec.sel.contain', 'rec.sel.cosrej', 'rec.sel.cvnloosepreselptp', 'rec.sel.remid', 'rec.sel.scann']>
rec.mc.nu <KeysViewHDF5 ['E', 'L', 'W2', 'batch', 'cycle', 'det', 'eff', 'evt', 'generator', 'genweight', 'hitnuc', 'hitnucp.E', 'hitnucp.px', 'hitnucp.py', 'hitnucp.pz', 'inttype', 'is0HC', 'isFHC', 'isRHC', 'iscc', 'ischarm', 'isseaquark', 'isvtxcont', 'mode', 'nhitslc', 'nhittot', 'nneutron', 'npiminus', 'npiplus', 'npizero', 'nproton', 'p.E', 'p.px', 'p.py', 'p.pz', 'pdg', 'pdgorig', 'pur', 'q2', 'rec.mc.nu_idx', 'resnum', 'run', 'subevt', 'subrun', 'tgtA', 'tgtZ', 'time', 'visE', 'visEBirks', 'visENeutron', 'visENeutronBirks', 'visEinslc', 'visEinslcBirks', 'visEinslcNeutron', 'visEinslcNeutronBirks', 'vtx.x', 'vtx.y', 'vtx.z', 'woscdumb', 'x', 'xsec', 'y']>
rec.mc.nu.beam <KeysViewHDF5 ['batch', 'cycle', 'dk2gen', 'dk2vtx', 'evt', 'gen2vtx', 'mupare', 'muparp.x', 'muparp.y', 'mup

KeyError: "Unable to open object (object 'rec.training.trainingdata' doesn't exist)"

In [5]:
print("Neutrino Final State code",df['rec.training.trainingdata']['finalstate'][3])
print("Interaction was ",Interaction(df['rec.training.trainingdata']['interaction'][3]))
print("Neutrino energy",df['rec.training.trainingdata']['nuenergy'][3],"GeV")

Neutrino Final State code [4]
Interaction was  Interaction.kNumuQE
Neutrino energy [0.839629] GeV


In [6]:
def getVariableArray(df,group,key):
    return np.array(df[group][key])[:,0]

trueVarList=[('rec.mc','nnu'),
         ('rec.mc.nu','pdg'),
         ('rec.mc.nu','mode'),
         ('rec.mc.nu','iscc'),
            ('rec.mc.nu','E')]

varList=[('rec.mc','nnu'),
         ('rec.mc.nu','pdg'),
         ('rec.mc.nu','mode'),
         ('rec.mc.nu','iscc'),
         ('rec.trk.kalman','idxremid'),
         ('rec.trk.kalman','ntracks'),
         ('rec.trk.kalman.tracks','dir.x'),
         ('rec.trk.kalman.tracks','dir.y'),
         ('rec.trk.kalman.tracks','dir.z'),
         ('rec.trk.kalman.tracks','nhit'),
         ('rec.trk.kalman.tracks','nplane'),
         ('rec.trk.kalman.tracks','maxplanecont'),
         ('rec.sel.remid','pid'),
         ('rec.sel.cvnloosepreselptp','numuid'),
         ('rec.sel.cvnloosepreselptp','nueid'),
         ('rec.sel.cvnloosepreselptp','ncid'),
         ('rec.sel.cvnloosepreselptp','nutauid'),
         ('rec.sel.cvnloosepreselptp','cosmicid'),
         ('rec.energy.numu','E'),
         ('rec.energy.numu','calccE'),
         ('rec.energy.numu','lstmnu'),
         ('rec.energy.numu','lstmmuon'),
         ('rec.energy.numu','regcvnhadE'),
         ('rec.energy.numu','hadcalE'),
         ('rec.energy.numu','hadtrkE'),
         ('rec.energy.numu.hadclust','calE'),
         ('rec.energy.numu.hadclust','nhit'),
         ('rec.mc.nu.beam','v.x'),
         ('rec.mc.nu.beam','v.y'),
         ('rec.mc.nu.beam','v.z'),
         ('rec.energy.numu','trkccE'),
         ('rec.sel.remid','pid'),
         ('rec.slc','nhit'), 
         ('rec.slc','ncontplanes'), 
         ('rec.trk.cosmic','ntracks'),
         ('rec.sel.cosrej','numucontpid2020fhc'),
         ('rec.sel.veto','keep'),
         ('rec.spill.cosmiccvn','passSel'),
         ('rec.slc','firstplane'),
         ('rec.slc','lastplane'),
         ('rec.sel.contain','kalfwdcell'),
         ('rec.sel.contain','kalbakcell'),
         ('rec.sel.contain','cosfwdcell'), 
         ('rec.sel.contain','cosbakcell')
        ]

def getVarDict(df,varList):
    thisDict={}
    for group,key in varList:
        #print(group,key)
        newkey=group+"."+key
        thisDict[newkey]=getVariableArray(df,group,key)
    return thisDict


td=getVarDict(df,trueVarList)


#for key in td:
#    print(key, np.shape(td[key]))

#print(np.sum(td['rec.mc.nnu']))

#print(td)

#idxremid=np.array(df['rec.trk.kalman']['idxremid'])[:,0]
#ntracks=np.array(df['rec.trk.kalman']['ntracks'])[:,0]
#remid=np.array(df['rec.sel.remid']['pid'])[:,0]
#numupid=np.array(df['rec.sel.cvnloosepreselptp']['numuid'])[:,0]
#nnuVals=np.array(df['rec.mc']['nnu'])[:,0]
#pdgVals=np.array(df['rec.mc.nu']['pdg'])[:,0]
#modeVals=np.array(df['rec.mc.nu']['mode'])[:,0]
#ccVals=np.array(df['rec.mc.nu']['iscc'])[:,0]
#lstmnu=np.array(df['rec.energy.numu']['lstmnu'])[:,0]
#lstmmuon=np.array(df['rec.energy.numu']['lstmmuon'])[:,0]
#hadCalE=np.array(df['rec.energy.numu.hadclust']['calE'])[:,0]
#hadHits=getVariableArray(df,'rec.energy.numu.hadclust','nhit')
#vz=np.array(df['rec.mc.nu.beam']['v.z'])[:,0]
#vy=np.array(df['rec.mc.nu.beam']['v.y'])[:,0]
#vx=np.array(df['rec.mc.nu.beam']['v.x'])[:,0]
#print(nnuVals.shape)
#print(nnuVals.shape)
#print(idxremid.shape)

#isNuMu=np.copy(nnuVals)
#isCC=np.copy(nnuVals)
#j=0
#for i in range(len(nnuVals)):
#    if nnuVals[i]:
#        #print(i,j,pdgVals[j],ccVals[j])
#        isNuMu[i] = True if (pdgVals[j]==14) else False
#        isCC[i] = (ccVals[j]) 
#        j=j+1
        
#print(isNuMu)
#print(isCC)


## This bit is pretty cool we take the number of (simuilated) neutrinos in each event/slice/whatever
## Then we find the cumulative sum of that array and use it is an index to pick out the correct neutrino truth information
def fillTruthVars(td):
    numNu=np.sum(td['rec.mc.nnu'])
    nnuIndex=np.cumsum(td['rec.mc.nnu'])
    nnuIndex=np.concatenate(([0],nnuIndex))
    nnuIndex[nnuIndex==numNu]=numNu-1  #If the last few entries are cosmics they
    print(nnuIndex[-10:]) #will try and access elements that don't exist
    td['nnuIndex']=nnuIndex
    td['pdgAll']=td['rec.mc.nu.pdg'][nnuIndex[0:-1]]*(td['rec.mc.nnu']>0.5) #The *(nnuVals>0) part sets all non-neutrinos (cosmics??) to zero for the pdg
    td['ccAll']=td['rec.mc.nu.iscc'][nnuIndex[0:-1]]*(td['rec.mc.nnu']>0.5)
    td['modeAll']=td['rec.mc.nu.mode'][nnuIndex[0:-1]]*(td['rec.mc.nnu']>0.5) #The *(nnuVals>0) part sets all non-neutrinos (cosmics??) to zero for the pdg
    td['trueEnu']=td['rec.mc.nu.E'][nnuIndex[0:-1]]*(td['rec.mc.nnu']>0.5)
    td['isNotNu']=td['rec.mc.nnu']<1
    td['isCC']=td['ccAll']>0
    td['isNC']=(td['rec.mc.nnu']*(1-td['isCC'])>0)
    td['isNuMu']=(td['pdgAll']==14)
    td['isNuMuCC']=(td['pdgAll']==14)*td['isCC']
    td['isANuMuCC']=(td['pdgAll']==-14)*td['isCC']
    td['isNuECC']=(td['pdgAll']==12)*td['isCC']
    td['isANuECC']=(td['pdgAll']==-12)*td['isCC']
                                        
                                         
fillTruthVars(td)


td.pop('rec.mc.nu.pdg')
td.pop('rec.mc.nu.mode')
td.pop('rec.mc.nu.iscc')
td.pop('rec.mc.nu.E')
td.pop('nnuIndex')


[25942169 25942169 25942170 25942170 25942171 25942171 25942171 25942171
 25942171 25942171]


array([       0,        0,        0, ..., 25942171, 25942171, 25942171])

# Back to my fun geometry stuff
In [DocDb-14341](https://nova-docdb.fnal.gov/cgi-bin/sso/ShowDocument?docid=14341) we describe a bug that was fixed in the NOvA detector geometry. We also provide conversion from the Far Detector local coordinate system (used in the FD siumulation) to the NuMI coordinate system (used in the beam simulation).


## Far Detector Transform
This transform is
$$r_{NuMI} =  M_{rot} ( r_{FD} - r_{offset})
$$
where
$$r_{offset} = \begin{pmatrix}57.666820 & \\ -51777.408547 \\ -808852.640542\end{pmatrix}$$
and
$$
M_{rot} =
\begin{pmatrix} 
  0.99990622174 & 0.000472822894411  & 0.0136866417481  \\
  0.000472822894411 & 0.997616062703 & -0.0690070132296 \\
  -0.0136866417481 &  0.0690070132296   &  0.997522284444     
\end{pmatrix}
$$

## Near Detector Transform
$$
r_{NuMI} =  M_{rot} ( r_{ND} - r_{offset})
$$
where
$$r_{offset} = \begin{bmatrix}2.269447 \\ 61.001882 \\-991.131313 \end{bmatrix}$$
and
$$
M_{rot} =
\begin{pmatrix} 
  0.99990 & 3.0533\times10^{-6}  & 1.4112\times10^{-2} \\
  -8.2300\times10^{-4} & 0.99831 & 5.8097\times10^{-2} \\
  -1.4088\times10^{-2} &    -5.8103\times10^{-2} &        0.99821
\end{pmatrix}
$$




In [7]:
#Taken from DocDb-14341
r_off_fd=np.array([57.666820,-51777.408547,-808852.640542]) 
m_rot_fd=np.array([[0.99990622174,0.000472822894411,0.0136866417481],
                   [0.000472822894411,0.997616062703,-0.0690070132296],
                  [-0.0136866417481,0.0690070132296,0.997522284444]])
m_rot_fd_inv=np.linalg.inv(m_rot_fd)


r_off_nd=np.array([2.269447 , 61.001882 ,-991.131313]) 
m_rot_nd=np.array([[0.99990 , 3.0533e-6,  1.4112e-2],
                   [8.2300e-4, 0.99831 , 5.8097e-2],
                  [-1.4088e-2 ,    -5.8103e-2 ,        0.99821]])
m_rot_nd_inv=np.linalg.inv(m_rot_nd)

#Block centres from the survey measurements in e.g. DocDb-13501
fdBlkCtr=np.array([11037.705436,-4164.619757,810452.13544])
ndBlkCtr=np.array([11.793636,-2.928869,999.282492])

pointS=np.array([0,0,170.399278])
mczero=np.array([0,0,0])
point100=np.array([0,0,100])
point600=np.array([0,0,600])
point800=np.array([0,0,800])

def convFDtoNuMI(r_fd):
    return np.matmul(m_rot_fd,r_fd-r_off_fd)

def convNuMItoFD(r_numi):
    return np.matmul(m_rot_fd_inv,r_numi)+r_off_fd

def convNDtoNuMI(r_nd):
    return np.matmul(m_rot_nd,r_nd-r_off_nd)

def convNuMItoND(r_numi):
    return np.matmul(m_rot_nd_inv,r_numi)+r_off_nd

r_fd=np.array([0,0,60])
print(convFDtoNuMI(r_fd))


def getDirAtDetectorFromPoint(whichDet,point):
    if whichDet=="FD":
        fdCentFromPoint=fdBlkCtr-point
        fdCentDist=np.linalg.norm(fdCentFromPoint)
        fdCfP=fdCentFromPoint/np.linalg.norm(fdCentFromPoint)
        r_fd_beamFront=convNuMItoFD(pointS+(fdCentDist-29.942)*fdCfP)
        r_fd_beamBack=convNuMItoFD(pointS+(fdCentDist+29.942)*fdCfP)
        r_fd_beamDir=r_fd_beamBack-r_fd_beamFront
        return r_fd_beamDir
    elif whichDet=="ND":
        ndCentFromPoint=ndBlkCtr-point
        ndCentDist=np.linalg.norm(ndCentFromPoint)
        ndCfP=ndCentFromPoint/np.linalg.norm(ndCentFromPoint)
        r_nd_beamFront=convNuMItoND(pointS+(ndCentDist-29.942)*ndCfP)
        r_nd_beamBack=convNuMItoND(pointS+(ndCentDist+29.942)*ndCfP)
        r_nd_beamDir=r_nd_beamBack-r_nd_beamFront
        return r_nd_beamDir
    else:
        print("Don't have detector",whichDet)
        
        
r_fd_beamDir=getDirAtDetectorFromPoint("FD",pointS)
r_fd_beamDirUnit=r_fd_beamDir/np.linalg.norm(r_fd_beamDir)
print("r_fd_beamDir",r_fd_beamDir)
print("r_fd_beamDirUnit",r_fd_beamDirUnit)
r_nd_beamDir=getDirAtDetectorFromPoint("ND",pointS)
r_nd_beamDirUnit=r_nd_beamDir/np.linalg.norm(r_nd_beamDir)

for point in [mczero,point100,pointS,point600,point800]:
    print(point)
    rp=getDirAtDetectorFromPoint("ND",point)
    rp/=np.linalg.norm(rp)
    print("\t",rp,180*math.atan(rp[0]/rp[2])/math.pi,180*math.atan(rp[1]/rp[2])/math.pi)
import math
print("r_nd_beamDir",r_nd_beamDir,180*math.atan(r_nd_beamDir[1]/r_nd_beamDir[2])/math.pi)
print("r_nd_beamDirUnit",r_nd_beamDirUnit)


[ 11038.11764868  -4166.69810204 810482.17869079]
r_fd_beamDir [-4.08908094e-03  3.82534312e+00  5.97616950e+01]
r_fd_beamDirUnit [-6.82833636e-05  6.38792185e-02  9.97957635e-01]
[0 0 0]
	 [-0.00228449 -0.06102058  0.99813389] -0.13113611583944212 -3.498404261662031
[  0   0 100]
	 [-9.72040557e-04 -6.13470174e-02  9.98116025e-01] -0.055798927645348176 -3.5171352997696474
[  0.         0.       170.399278]
	 [ 1.41822179e-04 -6.16239720e-02  9.98099427e-01] 0.008141285333660259 -3.5330320844841254
[  0   0 600]
	 [ 0.01544502 -0.0654208   0.99773823] 0.8868699638910319 -3.751462511134568
[  0   0 800]
	 [ 0.04501506 -0.07271388  0.99633646] 2.5868974399857207 -4.1741174416391935
r_nd_beamDir [ 8.49288940e-03 -3.69029431e+00  5.97702568e+01] -3.5330320844841254
r_nd_beamDirUnit [ 1.41822179e-04 -6.16239720e-02  9.98099427e-01]


In [8]:
trackVarList=[
         ('rec.trk.kalman','ntracks'),
         ('rec.trk.kalman.tracks','dir.x'),
         ('rec.trk.kalman.tracks','dir.y'),
         ('rec.trk.kalman.tracks','dir.z'),
         ('rec.energy.numu','lstmnu'),
         ('rec.energy.numu','lstmmuon')]

def fillVars(df,varList,thisDict):
    for group,key in varList:
        #print(group,key)
        newkey=group+"."+key
        thisDict[newkey]=getVariableArray(df,group,key)

fillVars(df,trackVarList,td)

def fillTrackKinematics(td):
    trackIndices=np.cumsum(td['rec.trk.kalman.ntracks'])
    trackIndices=np.concatenate(([0],trackIndices)).astype(int)
    td['trackIndices']=trackIndices
    td['cosBeam']=td['rec.trk.kalman.tracks.dir.x']*r_fd_beamDirUnit[0] + td['rec.trk.kalman.tracks.dir.y']*r_fd_beamDirUnit[1] + td['rec.trk.kalman.tracks.dir.z']*r_fd_beamDirUnit[2]
    M_p = 0.938272
    td['cosBeamFirst']=td['cosBeam'][trackIndices[0:-1]] #Magic line to select only those cos values that are from the first track
    td['ptp']=np.sqrt(1-td['cosBeamFirst']*td['cosBeamFirst'])
    td['pmu']=np.sqrt(td['rec.energy.numu.lstmmuon']*td['rec.energy.numu.lstmmuon'] - (105.658e-3)**2)
    td['pt']=td['ptp']*td['pmu']
    td['recoq2']=2*td['rec.energy.numu.lstmnu']*(td['rec.energy.numu.lstmmuon']-td['pmu']*td['cosBeamFirst'])-(105.658e-3)**2
    td['recow']=np.sqrt(M_p * M_p + 2 * M_p * (td['rec.energy.numu.lstmnu'] - td['rec.energy.numu.lstmmuon']) - td['recoq2'])
    
    
fillTrackKinematics(td)  
#Remove some unecessary variables
td.pop('rec.trk.kalman.tracks.dir.x')
td.pop('rec.trk.kalman.tracks.dir.y')
td.pop('rec.trk.kalman.tracks.dir.z')

#print(np.shape(cosBeam))
#print(np.shape(trackIndices))
#print(np.shape(ntracks))

/tmp/ipykernel_5924/1050324000.py:25: RuntimeWarning: invalid value encountered in sqrt
  td['pmu']=np.sqrt(td['rec.energy.numu.lstmmuon']*td['rec.energy.numu.lstmmuon'] - (105.658e-3)**2)
/tmp/ipykernel_5924/1050324000.py:28: RuntimeWarning: invalid value encountered in sqrt
  td['recow']=np.sqrt(M_p * M_p + 2 * M_p * (td['rec.energy.numu.lstmnu'] - td['rec.energy.numu.lstmmuon']) - td['recoq2'])


array([0.17169638, 0.80034024, 0.52732146, ..., 0.4417976 , 0.15074742,
       0.31009743], dtype=float32)

In [9]:
def getNumuBasicQuality(df):
    a=getVariableArray(df,'rec.energy.numu','trkccE') > 0 # nothing is terribly wrong
    b=getVariableArray(df,'rec.sel.remid','pid') > 0 # ensures at least 1 3D Kalman track with a remid value
    c=getVariableArray(df,'rec.slc','nhit') > 20         # low hits stuff is junk
    d=getVariableArray(df,'rec.slc','ncontplanes') > 4     # remove really vertical stuff
    e=getVariableArray(df,'rec.trk.cosmic','ntracks') > 0 # need a cosmic track
    return  a*b*c*d*e


def getNumu2020Pid(df):
    a=getVariableArray(df,'rec.sel.remid','pid')>0.30
    b=getVariableArray(df,'rec.sel.cvnloosepreselptp','numuid')>0.80
    return a*b

def getNumuBaseContainFD2020(df):
    a=getVariableArray(df,'rec.sel.contain','kalfwdcell') > 6
    b=getVariableArray(df,'rec.sel.contain','kalbakcell') > 6 
    c=getVariableArray(df,'rec.sel.contain','cosfwdcell') > 5 
    d=getVariableArray(df,'rec.sel.contain','cosbakcell') > 7
    e=getVariableArray(df,'rec.slc','firstplane')>2. #Not quite the right cut
    f=896-getVariableArray(df,'rec.slc','lastplane') >3 #Not quite the right cut
    return  a*b*c*d*e*f

def getNumu2020CosRej(td):
    return getVariableArray(df,'rec.sel.cosrej','numucontpid2020fhc')>0.45

def get3Flavour2020FDVeto(td):
    a=getVariableArray(df,'rec.sel.veto','keep')>0
    #b=td['rec.spill.cosmiccvn.passSel']>0
    #return a*b
    return a

def fillCutVars(df,td):
    td['numuBasicQuality']=getNumuBasicQuality(df)
    td['numuQuality']=(td['numuBasicQuality'])*(td['rec.energy.numu.lstmnu']<5)
    td['numu2020pid']=getNumu2020Pid(df)
    td['numucontain']=getNumuBaseContainFD2020(df)
    td['numucosrej']=getNumu2020CosRej(df)
    td['3flavourveto']=get3Flavour2020FDVeto(df)
    td['numufull']=td['numuQuality']*td['numu2020pid']*td['numucontain']*td['numucosrej']*td['3flavourveto'] 
    td['numunotpid']=td['numuQuality']*td['numucontain']*td['numucosrej']*td['3flavourveto']*(td['numu2020pid']<1)


fillCutVars(df,td)

for key in td:
    print(key,np.shape(td[key]))

rec.mc.nnu (83076140,)
pdgAll (83076140,)
ccAll (83076140,)
modeAll (83076140,)
trueEnu (83076140,)
isNotNu (83076140,)
isCC (83076140,)
isNC (83076140,)
isNuMu (83076140,)
isNuMuCC (83076140,)
isANuMuCC (83076140,)
isNuECC (83076140,)
isANuECC (83076140,)
rec.trk.kalman.ntracks (83076140,)
rec.energy.numu.lstmnu (83076140,)
rec.energy.numu.lstmmuon (83076140,)
trackIndices (83076141,)
cosBeam (120391912,)
cosBeamFirst (83076140,)
ptp (83076140,)
pmu (83076140,)
pt (83076140,)
recoq2 (83076140,)
recow (83076140,)
numuBasicQuality (83076140,)
numuQuality (83076140,)
numu2020pid (83076140,)
numucontain (83076140,)
numucosrej (83076140,)
3flavourveto (83076140,)
numufull (83076140,)
numunotpid (83076140,)


In [10]:
for key in td:
    print(key,np.shape(td[key]))

#temp=np.copy(td['isNuMuCC'])

#for cutName in ['numuBasicQuality','numuQuality','numucontain','numucosrej','3flavourveto','numu2020pid','numufull','numunotpid']:
#    print(cutName,np.sum(td[cutName]),np.sum(td[cutName]*td['isNuMuCC']),np.sum(temp))
#    temp*=td[cutName]


td.pop('cosBeam')
td.pop('trackIndices')



rec.mc.nnu (83076140,)
pdgAll (83076140,)
ccAll (83076140,)
modeAll (83076140,)
trueEnu (83076140,)
isNotNu (83076140,)
isCC (83076140,)
isNC (83076140,)
isNuMu (83076140,)
isNuMuCC (83076140,)
isANuMuCC (83076140,)
isNuECC (83076140,)
isANuECC (83076140,)
rec.trk.kalman.ntracks (83076140,)
rec.energy.numu.lstmnu (83076140,)
rec.energy.numu.lstmmuon (83076140,)
trackIndices (83076141,)
cosBeam (120391912,)
cosBeamFirst (83076140,)
ptp (83076140,)
pmu (83076140,)
pt (83076140,)
recoq2 (83076140,)
recow (83076140,)
numuBasicQuality (83076140,)
numuQuality (83076140,)
numu2020pid (83076140,)
numucontain (83076140,)
numucosrej (83076140,)
3flavourveto (83076140,)
numufull (83076140,)
numunotpid (83076140,)


array([        0,         1,         2, ..., 120391910, 120391911,
       120391912])

In [11]:
for key in td:
    print(key,np.shape(td[key]))

rec.mc.nnu (83076140,)
pdgAll (83076140,)
ccAll (83076140,)
modeAll (83076140,)
trueEnu (83076140,)
isNotNu (83076140,)
isCC (83076140,)
isNC (83076140,)
isNuMu (83076140,)
isNuMuCC (83076140,)
isANuMuCC (83076140,)
isNuECC (83076140,)
isANuECC (83076140,)
rec.trk.kalman.ntracks (83076140,)
rec.energy.numu.lstmnu (83076140,)
rec.energy.numu.lstmmuon (83076140,)
cosBeamFirst (83076140,)
ptp (83076140,)
pmu (83076140,)
pt (83076140,)
recoq2 (83076140,)
recow (83076140,)
numuBasicQuality (83076140,)
numuQuality (83076140,)
numu2020pid (83076140,)
numucontain (83076140,)
numucosrej (83076140,)
3flavourveto (83076140,)
numufull (83076140,)
numunotpid (83076140,)


In [12]:
outFile = h5py.File('FD-Nominal-FHC-Nonswap_copymerge/reallymini_notsel.h5','w')
magicNum=np.shape(td['pdgAll'])[0]
for key in td:
    arr=td[key]
    if 'rec.mc.nu' in key:
        continue
    if 'rec.trk.kalman.tracks' in key:
        arr=td[key][td['trackIndices'][0:-1]]
    if(np.shape(arr)[0]!=magicNum):
        print("Skipping",key,np.shape(arr)[0],magicNum)
        continue
    arr=arr[td['numunotpid']>0]
    print(key,np.shape(arr))
    dset = outFile.create_dataset(key, data=arr)


rec.mc.nnu (3470534,)
pdgAll (3470534,)
ccAll (3470534,)
modeAll (3470534,)
trueEnu (3470534,)
isNotNu (3470534,)
isCC (3470534,)
isNC (3470534,)
isNuMu (3470534,)
isNuMuCC (3470534,)
isANuMuCC (3470534,)
isNuECC (3470534,)
isANuECC (3470534,)
rec.trk.kalman.ntracks (3470534,)
rec.energy.numu.lstmnu (3470534,)
rec.energy.numu.lstmmuon (3470534,)
cosBeamFirst (3470534,)
ptp (3470534,)
pmu (3470534,)
pt (3470534,)
recoq2 (3470534,)
recow (3470534,)
numuBasicQuality (3470534,)
numuQuality (3470534,)
numu2020pid (3470534,)
numucontain (3470534,)
numucosrej (3470534,)
3flavourveto (3470534,)
numufull (3470534,)
numunotpid (3470534,)


TypeError: unhashable type: 'dict_keys'

In [15]:
keyList=list(td.keys())
for key in keyList:
    print(key)
    if key != 'numunotpid':
        if key != 'numufull':
            td.pop(key)

rec.mc.nnu
pdgAll
ccAll
modeAll
trueEnu
isNotNu
isCC
isNC
isNuMu
isNuMuCC
isANuMuCC
isNuECC
isANuECC
rec.trk.kalman.ntracks
rec.energy.numu.lstmnu
rec.energy.numu.lstmmuon
cosBeamFirst
ptp
pmu
pt
recoq2
recow
numuBasicQuality
numuQuality
numu2020pid
numucontain
numucosrej
3flavourveto
numufull
numunotpid


In [16]:
varList=[
         ('rec.sel.remid','pid'),
         ('rec.sel.cvnloosepreselptp','numuid'),
         ('rec.sel.cvnloosepreselptp','nueid'),
         ('rec.sel.cvnloosepreselptp','ncid'),
         ('rec.sel.cvnloosepreselptp','nutauid'),
         ('rec.sel.cvnloosepreselptp','cosmicid'),
         ('rec.energy.numu','E'),
         ('rec.energy.numu','calccE'),
         ('rec.energy.numu','regcvnhadE'),
         ('rec.energy.numu','hadcalE'),
         ('rec.energy.numu','hadtrkE'),
         ('rec.energy.numu.hadclust','calE'),
         ('rec.energy.numu.hadclust','nhit'),
         ('rec.slc','nhit'), 
         ('rec.slc','ncontplanes'), 
         ('rec.trk.cosmic','ntracks'),
        ('rec.sel.contain','cosbakcell')
        ]
for group,key in varList:
    print(group,key)
    arr=getVariableArray(df,group,key)[td['numunotpid']>0]
    dset = outFile.create_dataset(group+"."+key, data=arr)
outFile.close()

rec.sel.remid pid
rec.sel.cvnloosepreselptp numuid
rec.sel.cvnloosepreselptp nueid
rec.sel.cvnloosepreselptp ncid
rec.sel.cvnloosepreselptp nutauid
rec.sel.cvnloosepreselptp cosmicid
rec.energy.numu E
rec.energy.numu calccE
rec.energy.numu regcvnhadE
rec.energy.numu hadcalE
rec.energy.numu hadtrkE
rec.energy.numu.hadclust calE
rec.energy.numu.hadclust nhit
rec.slc nhit
rec.slc ncontplanes
rec.trk.cosmic ntracks
rec.sel.contain cosbakcell
